In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("/kaggle/input/gdsc-hongik-aiml-kaggle-comptetion/GDSC_train.csv")
df

In [13]:
from sklearn.model_selection import train_test_split

tot_features = df.drop(['default'], axis=1)
tot_labels = df['default']

X_train, X_val, y_train, y_val = train_test_split(tot_features, tot_labels, test_size=0.2)
print("👀Size of each datas...👀")
print(f"X_train : {X_train.shape}")
print(f"X_val : {X_val.shape}")
print(f"y_train : {y_train.shape}")
print(f"y_val : {y_val.shape}")

👀Size of each datas...👀
X_train : (19200, 23)
X_val : (4800, 23)
y_train : (19200,)
y_val : (4800,)


In [14]:
import xgboost as xgb

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

xgbc = xgb.XGBClassifier(objective='binary:logistic', random_state=42)
xgbc.fit(X_train, y_train)

feature_importance = xgbc.feature_importances_
columns = df.columns[:-1]

sorted_idx = feature_importance.argsort()[::-1]

# 중요도 순으로 정렬 후 출력
for idx in sorted_idx:
    print(f"{columns[idx]}: {feature_importance[idx]}")

# 최소 중요도를 가진 열 출력
print(f"Least important column: {columns[sorted_idx[-1]]}")


PAY_1: 0.33635038137435913
PAY_2: 0.11954481154680252
PAY_3: 0.05531349405646324
PAY_4: 0.043061502277851105
PAY_5: 0.03907407075166702
PAY_6: 0.029392952099442482
PAY_AMT2: 0.025603098794817924
BILL_AMT1: 0.025338396430015564
LIMIT_BAL: 0.025113165378570557
PAY_AMT3: 0.024696677923202515
PAY_AMT5: 0.02352241985499859
PAY_AMT1: 0.023116257041692734
BILL_AMT3: 0.022979477420449257
BILL_AMT4: 0.02255413867533207
BILL_AMT2: 0.02246742509305477
BILL_AMT5: 0.022102437913417816
EDUCATION: 0.022093741223216057
PAY_AMT4: 0.021953778341412544
PAY_AMT6: 0.021362800151109695
BILL_AMT6: 0.02022741548717022
MARRIAGE: 0.019211888313293457
AGE: 0.018052376806735992
SEX: 0.016867266967892647
Least important column: SEX


#### Scaler에 따른 성능 비교

In [18]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import accuracy_score
from sklearn import svm

scaler1 = StandardScaler()
X_train_std = scaler1.fit_transform(X_train)
X_val_std = scaler1.transform(X_val)

scaler2 = MinMaxScaler()
X_train_norm = scaler2.fit_transform(X_train)
X_val_norm = scaler2.transform(X_val)


scaler3 = RobustScaler()
X_train_rob = scaler3.fit_transform(X_train)
X_val_rob = scaler3.transform(X_val)

svm_model = svm.SVC()

# 원본 데이터
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_val)
print(f"Accuracy (original): {accuracy_score(y_val, y_pred)}")

# StandardScaler
svm_model.fit(X_train_std, y_train)
y_pred_std = svm_model.predict(X_val_std)
print(f"Accuracy (standardized): {accuracy_score(y_val, y_pred_std)}")

# MinMaxScaler
svm_model.fit(X_train_norm, y_train)
y_pred_norm = svm_model.predict(X_val_norm)
print(f"Accuracy (normalized): {accuracy_score(y_val, y_pred_norm)}")

# Robust
svm_model.fit(X_train_rob, y_train)
y_pred_rob = svm_model.predict(X_val_rob)
print(f"Accuracy (Robust): {accuracy_score(y_val, y_pred_rob)}")

Accuracy (original): 0.7791666666666667
Accuracy (standardized): 0.8214583333333333
Accuracy (normalized): 0.8222916666666666
Accuracy (Robust): 0.82125


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score


models = [
    LogisticRegression(solver='liblinear'),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(random_state=42),
    xgb.XGBClassifier(task_type='CPU', random_state=42),
    LGBMClassifier(random_state=42),
    CatBoostClassifier(task_type='CPU', random_seed=42, silent=True),
    KNeighborsClassifier()
]

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for model in models:
    print(f"{model.__class__.__name__}")
    scores = cross_val_score(model, X_scaled, y, cv=cv, scoring='accuracy')
    print(f"Accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})\n")


LogisticRegression
Accuracy: 0.8106 (+/- 0.0024)

SVC
Accuracy: 0.8205 (+/- 0.0049)

DecisionTreeClassifier
Accuracy: 0.7305 (+/- 0.0195)

RandomForestClassifier
Accuracy: 0.8150 (+/- 0.0041)

XGBClassifier
[07:46:17] WARNING: ../src/learner.cc:767: 
Parameters: { "task_type" } are not used.

[07:46:20] WARNING: ../src/learner.cc:767: 
Parameters: { "task_type" } are not used.

[07:46:23] WARNING: ../src/learner.cc:767: 
Parameters: { "task_type" } are not used.

[07:46:26] WARNING: ../src/learner.cc:767: 
Parameters: { "task_type" } are not used.

[07:46:29] WARNING: ../src/learner.cc:767: 
Parameters: { "task_type" } are not used.

Accuracy: 0.8152 (+/- 0.0069)

LGBMClassifier
Accuracy: 0.8215 (+/- 0.0046)

CatBoostClassifier
Accuracy: 0.8202 (+/- 0.0029)

KNeighborsClassifier
Accuracy: 0.7932 (+/- 0.0030)



### LGBMClassifier 사용해 학습 및 추론

In [17]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

lgbm = LGBMClassifier(random_state=42)

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
fold_accuracy = []

for train_idx, val_idx in cv.split(X_train_std, y_train):
    X_train_fold, X_val_fold = X_train_std[train_idx], X_train_std[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    lgbm.fit(X_train_fold, y_train_fold)
    y_val_pred = lgbm.predict(X_val_fold)
    val_accuracy = accuracy_score(y_val_fold, y_val_pred)
    fold_accuracy.append(val_accuracy)

print(f"Validation Accuracy per Fold: {fold_accuracy}")
print(f"Average Validation Accuracy: {np.mean(fold_accuracy):.4f}")

# 최종 모델 학습 및 테스트 데이터 추론
lgbm.fit(X_train_std, y_train)
y_val_pred = lgbm.predict(X_val_std)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"\nval Accuracy: {val_accuracy:.4f}")


Validation Accuracy per Fold: [0.8322916666666667, 0.8203125, 0.8161458333333333, 0.8174479166666667, 0.8197916666666667]
Average Validation Accuracy: 0.8212

val Accuracy: 0.8204


In [19]:
test_X = pd.read_csv('/kaggle/input/gdsc-hongik-aiml-kaggle-comptetion/GDSC_test.csv')
scaled_test_X = scaler.transform(test_X)

y_test_pred = lgbm.predict(scaled_test_X)

sub_preds = pd.DataFrame(y_test_pred, columns=['preds'])
sub_preds.to_csv('/kaggle/working/cv_lgbm_submission.csv', index_label='idx')
print("Done")

Done


In [20]:
scaler = MinMaxScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

lgbm = LGBMClassifier(random_state=42)

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
fold_accuracy = []

for train_idx, val_idx in cv.split(X_train_std, y_train):
    X_train_fold, X_val_fold = X_train_std[train_idx], X_train_std[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

    lgbm.fit(X_train_fold, y_train_fold)
    y_val_pred = lgbm.predict(X_val_fold)
    val_accuracy = accuracy_score(y_val_fold, y_val_pred)
    fold_accuracy.append(val_accuracy)

print(f"Validation Accuracy per Fold: {fold_accuracy}")
print(f"Average Validation Accuracy: {np.mean(fold_accuracy):.4f}")

# 최종 모델 학습 및 테스트 데이터 추론
lgbm.fit(X_train_std, y_train)
y_val_pred = lgbm.predict(X_val_std)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"\nval Accuracy: {val_accuracy:.4f}")

test_X = pd.read_csv('/kaggle/input/gdsc-hongik-aiml-kaggle-comptetion/GDSC_test.csv')
scaled_test_X = scaler.transform(test_X)

y_test_pred = lgbm.predict(scaled_test_X)

sub_preds = pd.DataFrame(y_test_pred, columns=['preds'])
sub_preds.to_csv('/kaggle/working/cv_lgbm_minmax_submission.csv', index_label='idx')
print("Done")

Validation Accuracy per Fold: [0.8286458333333333, 0.8213541666666667, 0.8143229166666667, 0.8166666666666667, 0.8203125]
Average Validation Accuracy: 0.8203

val Accuracy: 0.8196
Done


### torch 사용한 경우

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# 데이터 전처리
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

# 데이터 타입 변환
X_train_torch = torch.tensor(X_train_std, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.long)
X_val_torch = torch.tensor(X_val_std, dtype=torch.float32)
y_val_torch = torch.tensor(y_val, dtype=torch.long)

train_dataset = TensorDataset(X_train_torch, y_train_torch)
val_dataset = TensorDataset(X_val_torch, y_val_torch)

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 모델 설계
class SimpleClassifier(nn.Module):
    def __init__(self, input_features, output_classes):
        super(SimpleClassifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_features, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_classes),
        )

    def forward(self, x):
        return self.layers(x)


model = SimpleClassifier(X_train_torch.shape[1], len(np.unique(y_train_torch.numpy())))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 모델 학습
num_epochs = 100
for epoch in range(num_epochs):
    for xb, yb in train_dl:
        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")
with torch.no_grad():
    test_preds = model(X_val_torch)
    _, predicted_labels = torch.max(test_preds, dim=1)

    accuracy = (predicted_labels == y_val_torch).float().mean().numpy()
    print(f"\nTest Accuracy: {accuracy:.4f}")

Epoch [1/100], Loss: 0.5409
Epoch [11/100], Loss: 0.3558
Epoch [21/100], Loss: 0.4957
Epoch [31/100], Loss: 0.3725
Epoch [41/100], Loss: 0.2425
Epoch [51/100], Loss: 0.1096
Epoch [61/100], Loss: 0.1258
Epoch [71/100], Loss: 0.3029
Epoch [81/100], Loss: 0.2744
Epoch [91/100], Loss: 0.0968

Test Accuracy: 0.7754


In [12]:
test_X = pd.read_csv('/kaggle/input/gdsc-hongik-aiml-kaggle-comptetion/GDSC_test.csv')
scaled_test_X = scaler.transform(test_X)
X_test_torch = torch.tensor(scaled_test_X, dtype=torch.float32)

with torch.no_grad():
    test_preds = model(X_test_torch)
    _, predicted_labels = torch.max(test_preds, dim=1)
sub_preds = pd.DataFrame(predicted_labels, columns=['preds'])
sub_preds.to_csv('/kaggle/working/dl_submission.csv', index_label='idx')
print("Done")

Done


/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
